# 🚦 Vision Model Evaluator for Traffic Sign Recognition

This notebook provides a comprehensive toolkit for evaluating vision-language models on traffic sign recognition tasks.

## Supported Models
- **Google Gemini** (replaces deprecated Bard API)
- **OpenAI GPT-4V/GPT-4o**
- **LLaVA** (local deployment)

## Features
- ✅ Async processing for improved throughput
- ✅ Automatic retry with exponential backoff
- ✅ Progress tracking with tqdm
- ✅ Comprehensive logging
- ✅ Results export to CSV/JSON with metadata
- ✅ Batch processing with rate limiting
- ✅ Result visualization and analysis

## 📦 Installation

In [ ]:
# Install required packages
!pip install -q httpx tqdm pillow pandas matplotlib seaborn

# For Google Drive integration in Colab
from google.colab import drive
drive.mount('/content/drive')

## ⚙️ Configuration

In [ ]:
from dataclasses import dataclass, field, asdict
from pathlib import Path
from typing import Optional, Any
from enum import Enum
import os

class ModelType(str, Enum):
    GEMINI = "gemini"
    GPT4V = "gpt4v"
    LLAVA = "llava"

@dataclass
class Config:
    """Main configuration for the evaluation."""
    # Model settings
    model_type: ModelType = ModelType.GPT4V
    api_key: str = ""  # Set via environment or directly
    
    # Paths
    image_folder: str = "/content/drive/MyDrive/traffic_signs"
    output_folder: str = "/content/drive/MyDrive/results"
    
    # Evaluation settings
    prompt: str = (
        "Is the traffic sign displayed a real-world traffic sign that has the same "
        "shape, color, pattern and text as a real world traffic sign? "
        "Answer with 'yes' or 'no' and provide a brief explanation."
    )
    
    # Rate limiting
    delay_between_requests: float = 1.0  # seconds
    max_retries: int = 3
    
# Create config instance
config = Config()

# Set API key from environment or enter directly
# Option 1: From environment
config.api_key = os.environ.get('OPENAI_API_KEY', '') or os.environ.get('GEMINI_API_KEY', '')

# Option 2: Enter directly (uncomment and fill in)
# config.api_key = "your-api-key-here"

print(f"Model: {config.model_type.value}")
print(f"API Key configured: {'Yes' if config.api_key else 'No'}")
print(f"Image folder: {config.image_folder}")

## 🔧 Core Utilities

In [ ]:
import base64
import asyncio
import httpx
import time
import json
import csv
import logging
from datetime import datetime
from tqdm.notebook import tqdm
from abc import ABC, abstractmethod

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def encode_image(image_path: str) -> str:
    """Encode image to base64."""
    with open(image_path, "rb") as f:
        return base64.b64encode(f.read()).decode('utf-8')

def get_mime_type(path: str) -> str:
    """Get MIME type from file extension."""
    ext = Path(path).suffix.lower()
    return {
        '.jpg': 'image/jpeg',
        '.jpeg': 'image/jpeg',
        '.png': 'image/png',
        '.gif': 'image/gif',
        '.webp': 'image/webp'
    }.get(ext, 'image/jpeg')

def discover_images(folder: str) -> list[str]:
    """Find all images in folder."""
    extensions = ('.jpg', '.jpeg', '.png', '.gif', '.webp')
    folder_path = Path(folder)
    images = []
    for ext in extensions:
        images.extend(folder_path.glob(f'*{ext}'))
        images.extend(folder_path.glob(f'*{ext.upper()}'))
    return sorted([str(p) for p in set(images)])

@dataclass
class EvalResult:
    """Single evaluation result."""
    image_name: str
    prediction: str
    raw_response: str
    latency_ms: float
    success: bool
    error: Optional[str] = None
    timestamp: str = field(default_factory=lambda: datetime.now().isoformat())

print("✅ Utilities loaded")

## 🤖 Model Clients

In [ ]:
class BaseClient(ABC):
    """Base class for model clients."""
    
    def __init__(self, api_key: str, timeout: float = 60.0):
        self.api_key = api_key
        self.timeout = timeout
        self.client: Optional[httpx.AsyncClient] = None
    
    async def __aenter__(self):
        self.client = httpx.AsyncClient(timeout=self.timeout)
        return self
    
    async def __aexit__(self, *args):
        if self.client:
            await self.client.aclose()
    
    @abstractmethod
    async def analyze(self, image_path: str, prompt: str) -> str:
        pass


class GeminiClient(BaseClient):
    """Google Gemini API client."""
    
    MODEL = "gemini-1.5-flash"
    BASE_URL = "https://generativelanguage.googleapis.com/v1beta"
    
    async def analyze(self, image_path: str, prompt: str) -> str:
        image_data = encode_image(image_path)
        mime_type = get_mime_type(image_path)
        
        url = f"{self.BASE_URL}/models/{self.MODEL}:generateContent?key={self.api_key}"
        
        payload = {
            "contents": [{
                "parts": [
                    {"text": prompt},
                    {"inline_data": {"mime_type": mime_type, "data": image_data}}
                ]
            }],
            "generationConfig": {"temperature": 0.1, "maxOutputTokens": 100}
        }
        
        response = await self.client.post(url, json=payload)
        response.raise_for_status()
        data = response.json()
        
        return data["candidates"][0]["content"]["parts"][0]["text"]


class GPT4VClient(BaseClient):
    """OpenAI GPT-4 Vision client."""
    
    MODEL = "gpt-4o"  # or "gpt-4-vision-preview" for older API
    BASE_URL = "https://api.openai.com/v1"
    
    async def analyze(self, image_path: str, prompt: str) -> str:
        image_data = encode_image(image_path)
        mime_type = get_mime_type(image_path)
        
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }
        
        payload = {
            "model": self.MODEL,
            "messages": [{
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:{mime_type};base64,{image_data}",
                            "detail": "high"
                        }
                    }
                ]
            }],
            "max_tokens": 100,
            "temperature": 0.1
        }
        
        response = await self.client.post(
            f"{self.BASE_URL}/chat/completions",
            headers=headers,
            json=payload
        )
        response.raise_for_status()
        data = response.json()
        
        return data["choices"][0]["message"]["content"]


def get_client(model_type: ModelType, api_key: str) -> BaseClient:
    """Factory function to get appropriate client."""
    if model_type == ModelType.GEMINI:
        return GeminiClient(api_key)
    elif model_type == ModelType.GPT4V:
        return GPT4VClient(api_key)
    else:
        raise ValueError(f"Unsupported model: {model_type}")

print("✅ Model clients loaded")

## 🏃 Evaluation Runner

In [ ]:
class Evaluator:
    """Main evaluation runner."""
    
    def __init__(self, client: BaseClient, config: Config):
        self.client = client
        self.config = config
        self.results: list[EvalResult] = []
    
    def _extract_prediction(self, response: str) -> str:
        """Extract yes/no from response."""
        resp = response.lower().strip()
        if resp.startswith('yes'):
            return 'yes'
        elif resp.startswith('no'):
            return 'no'
        # Check for yes/no anywhere
        if resp.count('yes') > resp.count('no'):
            return 'yes'
        elif resp.count('no') > resp.count('yes'):
            return 'no'
        return 'unclear'
    
    async def evaluate_single(self, image_path: str) -> EvalResult:
        """Evaluate single image with retries."""
        image_name = Path(image_path).name
        start = time.perf_counter()
        
        for attempt in range(1, self.config.max_retries + 1):
            try:
                response = await self.client.analyze(image_path, self.config.prompt)
                latency = (time.perf_counter() - start) * 1000
                
                return EvalResult(
                    image_name=image_name,
                    prediction=self._extract_prediction(response),
                    raw_response=response,
                    latency_ms=latency,
                    success=True
                )
            except Exception as e:
                logger.warning(f"Attempt {attempt} failed for {image_name}: {e}")
                if attempt < self.config.max_retries:
                    await asyncio.sleep(2 ** attempt)  # Exponential backoff
                else:
                    latency = (time.perf_counter() - start) * 1000
                    return EvalResult(
                        image_name=image_name,
                        prediction='error',
                        raw_response='',
                        latency_ms=latency,
                        success=False,
                        error=str(e)
                    )
        
        raise RuntimeError("Should not reach here")
    
    async def run(self, images: list[str]) -> list[EvalResult]:
        """Run evaluation on all images."""
        for image_path in tqdm(images, desc="Evaluating"):
            result = await self.evaluate_single(image_path)
            self.results.append(result)
            await asyncio.sleep(self.config.delay_between_requests)
        
        return self.results
    
    def save_results(self, output_folder: str):
        """Save results to CSV and JSON."""
        Path(output_folder).mkdir(parents=True, exist_ok=True)
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        
        # CSV
        csv_path = Path(output_folder) / f"results_{timestamp}.csv"
        with open(csv_path, 'w', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=['image_name', 'prediction', 'raw_response', 'latency_ms', 'success', 'error'])
            writer.writeheader()
            for r in self.results:
                writer.writerow(asdict(r))
        
        # JSON
        json_path = Path(output_folder) / f"results_{timestamp}.json"
        with open(json_path, 'w') as f:
            json.dump({
                'metadata': {
                    'model': config.model_type.value,
                    'total_images': len(self.results),
                    'timestamp': timestamp
                },
                'results': [asdict(r) for r in self.results]
            }, f, indent=2)
        
        print(f"✅ Results saved to:\n   - {csv_path}\n   - {json_path}")
        return csv_path, json_path
    
    def summary(self) -> dict:
        """Get summary statistics."""
        successful = [r for r in self.results if r.success]
        predictions = [r.prediction for r in successful]
        latencies = [r.latency_ms for r in successful]
        
        return {
            'total': len(self.results),
            'successful': len(successful),
            'failed': len(self.results) - len(successful),
            'predictions': {
                'yes': predictions.count('yes'),
                'no': predictions.count('no'),
                'unclear': predictions.count('unclear')
            },
            'avg_latency_ms': sum(latencies) / len(latencies) if latencies else 0
        }

print("✅ Evaluator loaded")

## ▶️ Run Evaluation

In [ ]:
async def main():
    """Main evaluation pipeline."""
    # Validate config
    if not config.api_key:
        raise ValueError("Please set your API key in the config!")
    
    # Discover images
    images = discover_images(config.image_folder)
    if not images:
        raise ValueError(f"No images found in {config.image_folder}")
    
    print(f"📁 Found {len(images)} images")
    print(f"🤖 Using model: {config.model_type.value}")
    
    # Run evaluation
    async with get_client(config.model_type, config.api_key) as client:
        evaluator = Evaluator(client, config)
        await evaluator.run(images)
        
        # Save and summarize
        evaluator.save_results(config.output_folder)
        summary = evaluator.summary()
        
        print("\n📊 Summary:")
        print(json.dumps(summary, indent=2))
    
    return evaluator

# Run the evaluation
evaluator = await main()

## 📊 Visualization

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def visualize_results(evaluator: Evaluator):
    """Create visualizations of evaluation results."""
    df = pd.DataFrame([asdict(r) for r in evaluator.results])
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # Prediction distribution
    ax1 = axes[0]
    pred_counts = df['prediction'].value_counts()
    colors = {'yes': '#2ecc71', 'no': '#e74c3c', 'unclear': '#f39c12', 'error': '#95a5a6'}
    pred_counts.plot(kind='bar', ax=ax1, color=[colors.get(x, '#333') for x in pred_counts.index])
    ax1.set_title('Prediction Distribution')
    ax1.set_xlabel('Prediction')
    ax1.set_ylabel('Count')
    ax1.tick_params(axis='x', rotation=0)
    
    # Latency distribution
    ax2 = axes[1]
    successful = df[df['success'] == True]
    if not successful.empty:
        sns.histplot(successful['latency_ms'], bins=20, ax=ax2, color='#3498db')
        ax2.axvline(successful['latency_ms'].mean(), color='red', linestyle='--', label=f'Mean: {successful["latency_ms"].mean():.0f}ms')
        ax2.legend()
    ax2.set_title('Latency Distribution')
    ax2.set_xlabel('Latency (ms)')
    ax2.set_ylabel('Count')
    
    # Success rate
    ax3 = axes[2]
    success_counts = df['success'].value_counts()
    ax3.pie(
        success_counts.values,
        labels=['Success' if x else 'Failed' for x in success_counts.index],
        autopct='%1.1f%%',
        colors=['#2ecc71', '#e74c3c']
    )
    ax3.set_title('Success Rate')
    
    plt.tight_layout()
    plt.savefig(Path(config.output_folder) / 'results_visualization.png', dpi=150)
    plt.show()
    
    return df

# Visualize if we have results
if 'evaluator' in dir() and evaluator.results:
    df = visualize_results(evaluator)
    display(df.head(10))

---

# 🦙 LLaVA Local Setup

For running LLaVA locally (on Colab GPU or edge devices like Raspberry Pi).

In [ ]:
# Clone and setup LLaVA (for Colab with GPU)
!git clone -b dev https://github.com/camenduru/LLaVA /content/LLaVA
%cd /content/LLaVA

!pip install -q transformers==4.36.2 ninja
!pip install flash-attn --no-build-isolation
!pip install -e .

In [ ]:
# LLaVA Evaluation Script
LLAVA_EVAL_SCRIPT = '''
import os
import csv
import torch
from pathlib import Path
from tqdm import tqdm
from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN
from llava.conversation import conv_templates, SeparatorStyle
from llava.model.builder import load_pretrained_model
from llava.utils import disable_torch_init
from llava.mm_utils import tokenizer_image_token, KeywordsStoppingCriteria
from PIL import Image

class LLaVAEvaluator:
    def __init__(self, model_path: str, load_4bit: bool = True):
        disable_torch_init()
        self.tokenizer, self.model, self.image_processor, _ = load_pretrained_model(
            model_path, None, "llava", load_4bit=load_4bit
        )
        self.conv_mode = "llava_v1"
    
    def evaluate(self, image_path: str, prompt: str) -> str:
        image = Image.open(image_path).convert("RGB")
        image_tensor = self.image_processor.preprocess(image, return_tensors="pt")["pixel_values"].half().cuda()
        
        inp = DEFAULT_IMAGE_TOKEN + "\\n" + prompt
        conv = conv_templates[self.conv_mode].copy()
        conv.append_message(conv.roles[0], inp)
        conv.append_message(conv.roles[1], None)
        
        input_ids = tokenizer_image_token(
            conv.get_prompt(), self.tokenizer, IMAGE_TOKEN_INDEX, return_tensors="pt"
        ).unsqueeze(0).cuda()
        
        stop_str = conv.sep if conv.sep_style != SeparatorStyle.TWO else conv.sep2
        stopping_criteria = KeywordsStoppingCriteria([stop_str], self.tokenizer, input_ids)
        
        with torch.inference_mode():
            output_ids = self.model.generate(
                input_ids,
                images=image_tensor,
                do_sample=True,
                temperature=0.2,
                max_new_tokens=100,
                stopping_criteria=[stopping_criteria]
            )
        
        return self.tokenizer.decode(output_ids[0, input_ids.shape[1]:]).strip()
    
    def run_batch(self, image_folder: str, output_csv: str, prompt: str):
        images = list(Path(image_folder).glob("*.png")) + list(Path(image_folder).glob("*.jpg"))
        
        with open(output_csv, "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(["image_name", "prediction"])
            
            for img_path in tqdm(images, desc="Processing"):
                try:
                    result = self.evaluate(str(img_path), prompt)
                    writer.writerow([img_path.name, result])
                except Exception as e:
                    print(f"Error processing {img_path.name}: {e}")
                    writer.writerow([img_path.name, f"ERROR: {e}"])

if __name__ == "__main__":
    evaluator = LLaVAEvaluator("4bit/llava-v1.5-13b-3GB")
    evaluator.run_batch(
        "/content/images",
        "/content/llava_results.csv",
        "Is this a real traffic sign? Answer yes or no."
    )
'''

# Save the script
with open('/content/LLaVA/evaluate_traffic_signs.py', 'w') as f:
    f.write(LLAVA_EVAL_SCRIPT)

print("✅ LLaVA evaluation script saved")

In [ ]:
# Run LLaVA evaluation
%cd /content/LLaVA
!python evaluate_traffic_signs.py

---

# 🔧 Edge Device Deployment (llama.cpp)

For Raspberry Pi and NVIDIA Jetson devices.

In [ ]:
# Build llama.cpp for edge deployment
!git clone https://github.com/ggerganov/llama.cpp /content/llama.cpp
%cd /content/llama.cpp
!mkdir -p build && cd build && cmake .. && cmake --build . --config Release -j4

In [ ]:
# Edge deployment shell script
EDGE_SCRIPT = '''#!/bin/bash
# LLaVA Edge Evaluation Script
# For Raspberry Pi / Jetson / ZED Box

set -e

# Configuration
LLAVA_CLI="./build/bin/llava-cli"
MODEL="ggml-model-q4_k.gguf"
MMPROJ="mmproj-model-f16.gguf"
IMAGE_DIR="${1:-./images}"
OUTPUT_CSV="${2:-./results.csv}"
PROMPT="Is this a real traffic sign? Answer yes or no."

# Check dependencies
if [ ! -f "$LLAVA_CLI" ]; then
    echo "Error: llava-cli not found. Build llama.cpp first."
    exit 1
fi

if [ ! -f "$MODEL" ]; then
    echo "Downloading model from HuggingFace..."
    wget -q https://huggingface.co/jartine/llava-v1.5-7B-GGUF/resolve/main/ggml-model-q4_k.gguf
    wget -q https://huggingface.co/jartine/llava-v1.5-7B-GGUF/resolve/main/mmproj-model-f16.gguf
fi

# Initialize CSV
echo "image_name,prediction,raw_output" > "$OUTPUT_CSV"

# Process images
total=$(find "$IMAGE_DIR" -type f \( -name "*.png" -o -name "*.jpg" \) | wc -l)
count=0

for img in "$IMAGE_DIR"/*.{png,jpg,jpeg}; do
    [ -f "$img" ] || continue
    
    count=$((count + 1))
    basename=$(basename "$img")
    echo "[$count/$total] Processing: $basename"
    
    # Run inference
    output=$($LLAVA_CLI -m "$MODEL" --mmproj "$MMPROJ" --image "$img" -p "$PROMPT" 2>/dev/null || echo "ERROR")
    
    # Extract yes/no
    if echo "$output" | grep -iq "^yes"; then
        prediction="yes"
    elif echo "$output" | grep -iq "^no"; then
        prediction="no"
    else
        prediction="unclear"
    fi
    
    # Escape output for CSV
    escaped_output=$(echo "$output" | tr "\n" " " | sed 's/"/""/g')
    
    echo "$basename,$prediction,\"$escaped_output\"" >> "$OUTPUT_CSV"
done

echo "\n✅ Evaluation complete. Results saved to $OUTPUT_CSV"
'''

with open('/content/llama.cpp/run_evaluation.sh', 'w') as f:
    f.write(EDGE_SCRIPT)

!chmod +x /content/llama.cpp/run_evaluation.sh
print("✅ Edge deployment script saved")
print("\nUsage: ./run_evaluation.sh /path/to/images /path/to/output.csv")